In [1]:
# read in known applicant files, clean and try to dedupe
from collections import Counter
import difflib
from fuzzywuzzy import fuzz
import uuid
import itertools
import pandas as pd
import numpy as np
import string
import funcy
import re
import os


from data_cleaning_functions import correct_mispellings, long_form_date, clean_names, clean_med_school
from dev import (
    SUFFIXES, FEMALE_FIRST_NAMES, FEMALE_MIDDLE_NAMES, NAME_COLS, RAW_ATT_DATA_DIR, ATT_DATA_DIR, PICKLE_DIR, 
        CORRECTIONS_DIR, SUM_STAT_DIR)


%load_ext autoreload
%autoreload 2

%reload_ext autoreload

In [2]:
r1_file = '1964-1973 associates.XLS'
r2_file = 'Associates alpha by institute.XLS'
r3_file = 'Associates data.XLS'
r4_file = 'NIMH Associates Complete.XLS'
r5_file = 'NINDB Associates alpha by year.xls'
filenames = [r1_file, r2_file, r3_file, r4_file, r5_file]

In [3]:
file_df = map(lambda x: pd.read_excel(os.path.join(RAW_ATT_DATA_DIR, x)), filenames)

file_4_columns = [
    'dno', 'source', 'unknown', 'lastname', 'first_middle', 'institute', 'lab_brch', 
    'program', 'supervisor', 'eod_year', 'med_school', 'year_grad', 'intern_hos', 'intern_dte',
       'res_hosp', 'residency', 'res_dtes'
]

file_df[2].rename(columns={'lname':'lastname', 'fname': 'first_middle'}, inplace=True)
file_df[1].rename(columns={'lname':'lastname', 'fname': 'first_middle'}, inplace=True)

file_df[4].columns = file_4_columns
# for each files in the list, add a column to track source
for name, f in zip(filenames, file_df):
    f.loc[:, 'data_source'] = name
concat_df = pd.concat(file_df).reset_index(drop=True)

print sum(map(lambda x: x.shape[0], file_df)) == concat_df.shape[0]

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
True


In [4]:
def has_suffix(lst_strs):
    # check for any existing suffix
    existing_suffixes = filter(None, filter(lambda x: x in lst_strs, SUFFIXES))
    # if suffix is I or V, only return suffix if len of lst_string > 2 (otherwise middle name)
    if ('I' in existing_suffixes or 'V' in existing_suffixes) and len(lst_strs) > 2:
        return existing_suffixes
    return funcy.remove(lambda x: x in ['I', 'V', 'DR'], existing_suffixes)

In [5]:
def strip_first_middle(raw_str):
    # seperate first_middle column string (Thomas, Bruce) into THOMAS BRUCE
    # suffix also appears to be in column THOMAS BRUCE JR
    names = filter(None, raw_str.split(' '))
    cleaned_vals = map(clean_names, names)
    suff = has_suffix(cleaned_vals)
    # suffix is actually a list
    if suff:
        root = funcy.remove(lambda x: x==suff[-1], cleaned_vals)
        return pd.Series(
            {'clean_first_name': root[0],
                 'clean_middle_name': ' '.join(root[1:]),
                    'clean_suffix': suff[-1]})
    return pd.Series(
        {'clean_first_name': cleaned_vals[0],
             'clean_middle_name': np.nan if len(cleaned_vals)==1 else ' '.join(cleaned_vals[1:]),
                'clean_suffix': np.nan})

In [6]:
cleaned_suffix_df = pd.DataFrame(concat_df.loc[~pd.isnull(concat_df.first_middle), 'first_middle'].apply(strip_first_middle))

In [7]:
def get_suffix(raw_str):
    # wrapper around get suffix
    if pd.isnull(raw_str) or len(raw_str.split(' ')) < 2:
        return np.nan
    str_list = raw_str.split(' ')
    suff = has_suffix(str_list)
    if suff:
        return suff[-1]
    return np.nan

In [8]:
# now we have all the associates, sep first middle into first and middle name, then sort and check 
# to see if we have any duplicates
df2 = pd.concat([concat_df, cleaned_suffix_df], axis=1)
df2.head()
df2.loc[:, 'clean_last_name'] = df2.lastname.apply(clean_names)

In [9]:
# consolidate firstname columns
missing_first_middle = pd.isnull(df2.first_middle)
df2.loc[missing_first_middle, ['clean_first_name', 'clean_middle_name']] = df2.loc[missing_first_middle, ['firstname', 'middlename']].applymap(clean_names)
df2.loc[:, 'len_middle'] = df2.clean_middle_name.apply(lambda x: np.nan if pd.isnull(x) else len(x.split(' ')))

# clean suffix out from any other columns
may_have_suffix_mask = df2.len_middle > 1
df2.loc[may_have_suffix_mask, 'clean_suffix'] = df2[may_have_suffix_mask]['clean_middle_name'].apply(get_suffix)

df3 = df2.drop(['first_middle', 'firstname', 'middlename', 'lastname', 'len_middle'], axis=1)
# df3 = df2
# # dropnow where both first and last name are missing
df4 = df3.dropna(subset=['clean_first_name', 'clean_last_name'], how='all')
print df3.shape
print df4.shape

(10729, 24)
(10723, 24)


In [10]:
# remove all females
female_mask = np.logical_or(
    df4.clean_first_name.isin(FEMALE_FIRST_NAMES), df4.clean_middle_name.isin(FEMALE_MIDDLE_NAMES))

df5 = df4.loc[~female_mask, :].sort_values(NAME_COLS)
df5.loc[:, 'clean_middle_initial'] = df5.clean_middle_name.apply(lambda x: np.nan if pd.isnull(x) or x=='' else x[0])
df5.loc[:, 'clean_first_initial'] = df5.clean_first_name.apply(lambda x: np.nan if pd.isnull(x) or x=='' else x[0])
df5.loc[:, 'clean_medical_school'] = df5.med_school.apply(funcy.rcompose(clean_names, clean_med_school))

In [11]:
def count_missing(row):
    # count the number of entries that are null
    return row[pd.isnull(row)].size

df5.loc[:, 'count_missing'] = df5[
    NAME_COLS+['clean_medical_school', 'eod_year', 'res_dtes', 'res_hosp', 'year_grad']].apply(count_missing, axis=1)

In [12]:
# drop duplicates by dno
df6 = df5.drop_duplicates('dno').sort_values(
    ['clean_last_name', 'clean_first_initial', 'clean_middle_name', 'count_missing'], axis=0)
print df5.shape
print df6.shape

(10050, 28)
(3872, 28)


In [13]:
df6.loc[
    df6.clean_last_name=='HARLEY', NAME_COLS+['clean_medical_school', 'eod_year', 'res_dtes', 'res_hosp', 'year_grad']]

,clean_first_name,clean_middle_name,clean_last_name,clean_medical_school,eod_year,res_dtes,res_hosp,year_grad
3979,JOHN,B,HARLEY,NaN,NaN,NaN,NaN,NaN


In [14]:
def get_year_diff(x):
    # get reference value
    ref_year = x.iloc[0]
    return pd.Series(x - ref_year)

def is_duplicate(grouped_df):
    # return true if grouped df should be counted as one duplicate, returns filter mask
    # check if med schools match/are missing
    # check if eod years match/are missing
    # first group_id
    if grouped_df.shape[0] == 1:
        return pd.Series([np.nan])
    med_schools = grouped_df['clean_medical_school'].dropna().unique()
    # if only <= 1 unique med school, mark as duplicate
    ref_dno = grouped_df['dno'].values[0]
    if len(med_schools) < 2:
        return pd.Series([ref_dno]*grouped_df.shape[0])
    ms1 = med_schools[0]
    # otherwise, check the string similarity of the med school
    med_school_sims = [
        np.nan if pd.isnull(ms) else fuzz.partial_token_sort_ratio(
                ms1, ms) for ms in grouped_df.clean_medical_school.values]
    # for med schools with high string sim, count as dups, otherwise not 
    is_dups = [ref_dno if pd.isnull(sim) or sim > .8 else False for sim in med_school_sims]
    return pd.Series(is_dups)
    

In [15]:
df6.loc[df6.dno==4130, :]
df6.loc[df6.clean_last_name=='ZIVIN', :]

,citizenship,data_source,dno,dob,eod_year,generation,institute,intern_dte,intern_hos,lab_brch,...,unknown,year_grad,clean_first_name,clean_middle_name,clean_suffix,clean_last_name,clean_middle_initial,clean_first_initial,clean_medical_school,count_missing
5556,NaN,Associates alpha by institute.XLS,4130,NaN,1973.0,NaN,NIMH,NaN,NaN,NaN,...,NaN,1970.0,JUSTIN,ALLEN,NaN,ZIVIN,A,J,NORTHWESTERN,2
5557,NaN,Associates alpha by institute.XLS,4131,NaN,NaN,NaN,NIMH,NaN,NaN,NaN,...,NaN,1970.0,JUSTIN,ALLEN,NaN,ZIVIN,A,J,NORTHWESTERN,3


In [16]:
df6_grouped = df6.groupby(['clean_last_name', 'clean_first_initial'])
df6['eod_year_diff'] = df6_grouped.eod_year.apply(get_year_diff)
# df6['is_dup_index'] = df6_grouped.apply(is_duplicate)
new_col = df6_grouped.apply(is_duplicate)
# reset so no longer grouped as a multi index, then merge in via clean first, last_name

# dups = df6_grouped.filter(lambda x: is_duplicate(x)) 

In [17]:
new_col2 = new_col.reset_index()
new_col2.drop('level_2', axis=1, inplace=True)
print new_col.shape
print new_col2.shape


(3773,)
(3773, 3)


In [18]:
new_col2.columns = ['clean_last_name', 'clean_first_initial', 'duplicate_dno']
new_col3 = new_col2.drop_duplicates(['clean_last_name', 'clean_first_initial', 'duplicate_dno'])

In [19]:
new_col4 = new_col3[new_col3['duplicate_dno']!=False]
new_col4[new_col4['clean_last_name']=='AARON']

,clean_last_name,clean_first_initial,duplicate_dno
0,AARON,R,NaN


In [20]:
df7 = pd.merge(left=df6, right=new_col4, on=['clean_last_name', 'clean_first_initial'], how='inner')

In [21]:
df7.loc[:, 'eod_year_diff'] = df7.eod_year_diff.abs()
df7.loc[(df7.eod_year_diff>2) , 'duplicate_dno'] = np.nan

In [22]:
df7.loc[:, 'dup_flag'] = 0
dup_mask = df7.dno.isin(df7.duplicate_dno.dropna())
df7.loc[dup_mask, 'dup_flag'] = 1

In [23]:
missing_flags = ['missing_{}'.format(f) for f in df7.columns]

def count_missing(x):
    # returns 1 for missing value
    if x is None or x in ['None', '', ' ']:
        return 1
    return 1 if pd.isnull(x) else 0

# calculate missing info stats
df7[missing_flags] = df7.applymap(count_missing)

# calculate missing stats for people in the data set
df7[missing_flags].describe().to_csv(os.path.join(SUM_STAT_DIR, 'NIH_attendees_data_fill_rates.csv'))
df8 = df7.drop(missing_flags, axis=1)

In [24]:
def consolidate_row(series):
    nonnulls = series.dropna().unique()
    if nonnulls.shape[0] == 0:
        return pd.Series([np.nan]*series.shape[0])
    if isinstance(nonnulls[0], str):
        val = sorted(nonnulls, key=len, reverse=True)[0]
        return pd.Series([val]*series.shape[0])
    val = sorted(nonnulls, reverse=True)[0]
    return pd.Series([val]*series.shape[0])


In [25]:
grouped_dups = df8.loc[
    df8.duplicated(['clean_last_name', 'clean_first_initial'], keep=False), :].groupby(
        ['clean_last_name', 'clean_first_initial'], as_index=False)
new_df = []
for g, df in grouped_dups:
    consol_df = df.apply(consolidate_row, axis=0)
    consol_df['duplicate_dno'] = [df.dno.values]*df.shape[0]
    new_df.append(consol_df)
    

In [26]:
new_df2 = pd.concat(new_df, axis=0)
new_df3 = new_df2.drop_duplicates(['clean_first_name', 'clean_last_name', 'eod_year'])

In [27]:
#now, merge in manual corrections to add any more duplicate dnos
# finally consolidate people merged into the same row
df9 = pd.concat([df8.loc[pd.isnull(df8.duplicate_dno), :], new_df3], axis=0)


# df7.loc[df7.duplicated(['clean_last_name', 'clean_medical_school'], keep=False), NAME_COLS+['clean_medical_school', 'eod_year', 'dno', 'duplicate_dno']]

In [28]:
to_remove = ['TERRECE', 'FRED', 'LAURENCE',
             'CUONO', 'DEFRENZE', 'JEFFERY', 'FINKLEMAN', 'SHERRAD', 'ANSCHNETZ', 'MARC', 'JENSON', 'KASTI', 
            'ADELBERT', 'RITCHARD', 'MANSFORD', 'DEFRENZO', 'DROBIN', 'HAMES', 'KREUZ', 'JERROLD', 'MANEUSI',
            'UNGARO']
to_replace = ['TERRENCE', 'FREDERICK', 'LAWRENCE',
              'CUOMO', 'DEFRONZO', 'JEFFREY', 'FINKELMAN', 'SHERRARD', 'ANSCHUETZ', 'MARCUS', 'JENSEN', 'KASTL',
              'ALBERT', 'RITCHARD', 'MANIFORD', 'DEFRONZO', 'DROBIS', 'JAMES', 'KRUEZ', 'JERROD', 'MANCUSI',
              'UNGARO']

correct_name_mispellings_fnc = funcy.rpartial(correct_mispellings, to_remove, to_replace)

df9.loc[:, 'clean_last_name'] = df9.clean_last_name.apply(correct_name_mispellings_fnc)
df9.loc[:, 'clean_first_name'] = df9.clean_first_name.apply(correct_name_mispellings_fnc)

In [29]:
# function to go in and correct some of the name mispellings in both data sets
# MUTATING FUNCTION
def change_names(df, selection_type, selection_value, to_change_type, to_change_values):
    for t, v in zip(to_change_type, to_change_values):
        print t, v
        df.loc[df[selection_type]==selection_value, t] = v

In [30]:
change_names(
    df9, 'clean_last_name', 'CHESEBRO', ['clean_first_name', 'clean_middle_name'], ['BRUCE', 'WILCOX'])
change_names(df9, 'clean_last_name', 'GALANTER', ['clean_first_name', 'clean_middle_name'], ['MARC', 'I'])
change_names(
    df9, 'clean_last_name', 'BEAN', ['clean_first_name', 'clean_middle_name', 'clean_medical_school'], ['SIDNEY', 'CHARLES', 'WAKE_FOREST'])
change_names(
    df9, 'clean_last_name', 'EILER', ['clean_first_name', 'clean_middle_name'], ['DONALD', 'MARTIN'])
change_names(
    df9, 'clean_last_name', 'FALCHUK', ['clean_first_name', 'clean_middle_name'], ['DONALD', 'MARTIN'])


clean_first_name BRUCE
clean_middle_name WILCOX
clean_first_name MARC
clean_middle_name I
clean_first_name SIDNEY
clean_middle_name CHARLES
clean_medical_school WAKE_FOREST
clean_first_name DONALD
clean_middle_name MARTIN
clean_first_name DONALD
clean_middle_name MARTIN


In [31]:
# fill in missing eod year
# manual corrections
df9.loc[(df9.clean_last_name=='BEELS') & pd.isnull(df9.eod_year), 'eod_year'] = 1967
df9.loc[(df9.clean_last_name=='KRAUSE') & pd.isnull(df9.eod_year), 'eod_year'] = 1965
df9.loc[(df9.clean_last_name=='MELLMAN') & pd.isnull(df9.eod_year), 'eod_year'] = 1986
# df9.loc[pd.isnull(df9.eod_year), :]

In [32]:
# import manual eod year fixes
manual_eod_fixes = pd.read_excel(os.path.join(CORRECTIONS_DIR, 'manual_eod_fixes.xlsx'))
manual_eod_fixes.head()
manual_eod_df = manual_eod_fixes[['clean_last_name'] + [c for c in manual_eod_fixes.columns if c.startswith('to_fix')]]

missing_eod = pd.merge(
    left=df9.loc[pd.isnull(df9.eod_year), :], right=manual_eod_df, on='clean_last_name', how='inner')

for x in ['clean_medical_school', 'clean_first_name', 'clean_middle_name', 'eod_year']:
    mask = ~pd.isnull(missing_eod['to_fix_{}'.format(x)])
    missing_eod.loc[mask, x] = missing_eod.loc[mask, 'to_fix_{}'.format(x)]


In [33]:
missing_eod2 = missing_eod.drop(
    (c for c in missing_eod.columns if c not in df9.columns), axis=1).drop_duplicates(['clean_last_name'])

In [34]:
df10 = pd.concat([df9.loc[~pd.isnull(df9.eod_year), :], missing_eod2], axis=0)

In [35]:
missing_eod2[NAME_COLS+['eod_year', ]]

,clean_first_name,clean_middle_name,clean_last_name,eod_year
0,DONALD,NaN,ABELE,NaN
1,MARTIN,DAVID,ABELOFF,1967.0
2,GARY,W,CAGE,1963.0
3,MING,TA,CHONG,1977.0
4,GERALD,A M,FINERMAN,1965.0
5,PETER,NaN,GOLDMAN,1965.0
6,ERIC,NaN,GROSSMAN,NaN
7,JOHN,B,HARLEY,NaN
8,JAMES,MICHAEL,HERZOG,1973.0
9,GEORGE,H,HOLSTEN,1967.0


In [36]:
# it seems that dno does refer to unique person, so drop dups based on that 
# save this unique to pick
df10.to_pickle(os.path.join(PICKLE_DIR, 'unique_attendees.p'))

# to csv
df10.to_csv(os.path.join(ATT_DATA_DIR, 'unique_attendees.csv'))

In [37]:
missing_eod_year = df10.loc[pd.isnull(df10.eod_year), :].sort_values('clean_last_name')

missing_eod_year[NAME_COLS+['clean_medical_school', 'dno']].to_csv(os.path.join(ATT_DATA_DIR, 'missing_eod_year.csv'), index=False)